# Compile best.onnx → best.hef for Hailo-8 (26 TOPS)

Run this notebook on **Google Colab** (free, x86-64 Linux).

Steps:
1. Upload `best.onnx` and the Hailo DFC wheel to this notebook
2. Run all cells
3. Download `best.hef` and copy it to the Pi project directory

> **Get the DFC wheel:** https://hailo.ai/developer-zone/  
> Create a free account → Downloads → Hailo Dataflow Compiler → `.whl` for Python 3.10

In [ ]:
# ── Cell 1: Upload files ─────────────────────────────────────────────────────
from google.colab import files

print('Upload best.onnx')
uploaded = files.upload()   # select best.onnx

print('\nUpload hailo_dataflow_compiler-*.whl')
uploaded_whl = files.upload()   # select the .whl from Hailo Developer Zone

In [ ]:
# ── Cell 2: Install Hailo DFC ────────────────────────────────────────────────
import glob, subprocess, sys

whl = glob.glob('hailo_dataflow_compiler-*.whl')[0]
print(f'Installing {whl} …')
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', whl])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q',
                        'opencv-python-headless', 'numpy'])
print('Done.')

In [ ]:
# ── Cell 3: Compile ──────────────────────────────────────────────────────────
from hailo_sdk_client import ClientRunner

MODEL_NAME  = 'best'
HW_ARCH     = 'hailo8'     # Hailo-8 = 26 TOPS
NUM_CLASSES = 2             # 0=Plant  1=weeds
MAX_DETS    = 100

ALLS = f"""
normalization1 = normalization([0.0, 0.0, 0.0], [255.0, 255.0, 255.0])
nms_postprocess("", max_proposals_per_class={MAX_DETS}, classes={NUM_CLASSES}, \\
                regression_length=15, \\
                background_removal=False, background_removal_index=0)
"""

# 1 — Parse
print('[1/3] Parsing ONNX …')
runner = ClientRunner(hw_arch=HW_ARCH)
runner.translate_onnx_model(
    'best.onnx', MODEL_NAME,
    net_input_shapes={'images': [1, 3, 640, 640]},
    alls_script=ALLS)
runner.save_har('best_parsed.har')
print('    Saved best_parsed.har')

# 2 — Optimise (random calibration — upload real images for better accuracy)
print('[2/3] Optimising …')
runner = ClientRunner(hw_arch=HW_ARCH, har='best_parsed.har')
runner.optimize_full_precision()     # swap for runner.optimize(calib_data) if you have images
runner.save_har('best_optimized.har')
print('    Saved best_optimized.har')

# 3 — Compile
print('[3/3] Compiling to HEF …')
runner = ClientRunner(hw_arch=HW_ARCH, har='best_optimized.har')
hef_bytes = runner.compile()
with open('best.hef', 'wb') as f:
    f.write(hef_bytes)
print(f'    Saved best.hef  ({len(hef_bytes)/1024/1024:.1f} MB)')

In [ ]:
# ── Cell 4: Download best.hef ────────────────────────────────────────────────
from google.colab import files
files.download('best.hef')
print('Copy best.hef to the Pi project directory, then run:')
print('  python weed_detection_hailo.py')